##### decoder positional encoding 확장 X

In [1]:
import torch
from torch import nn
from transformers import BartForConditionalGeneration, BartConfig
from transformers.models.longformer.modeling_longformer import LongformerSelfAttention
from transformers.models.bart.modeling_bart import BartLearnedPositionalEmbedding
from typing import List, Optional, Tuple
from transformers import PreTrainedTokenizerFast

In [2]:
# Longformer Attention 레이어 정의
class LongformerAttention(nn.Module):
    def __init__(self, config, layer_id):
        super().__init__()
        self.embed_dim = config.d_model
        self.longformer_self_attn = LongformerSelfAttention(config, layer_id=layer_id)
        self.output_projection = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
        key_value_states: Optional[torch.Tensor] = None,
        past_key_value: Optional[Tuple[torch.Tensor]] = None,
        attention_mask: Optional[torch.Tensor] = None,
        layer_head_mask: Optional[torch.Tensor] = None,
        output_attentions: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:

        if attention_mask is not None:
            attention_mask = attention_mask.squeeze(dim=1)
            attention_mask = attention_mask[:, 0]

        is_index_masked = attention_mask < 0 if attention_mask is not None else None
        is_index_global_attn = attention_mask > 0 if attention_mask is not None else None
        is_global_attn = is_index_global_attn.flatten().any().item() if attention_mask is not None else False

        outputs = self.longformer_self_attn(
            hidden_states,
            attention_mask=attention_mask,
            layer_head_mask=layer_head_mask,
            is_index_masked=is_index_masked,
            is_index_global_attn=is_index_global_attn,
            is_global_attn=is_global_attn,
            output_attentions=output_attentions,
        )

        attention_output = self.output_projection(outputs[0])
        return (attention_output,) + outputs[1:] if len(outputs) == 2 else (attention_output, None, None)

In [3]:
# Longformer Attention 적용 KoBART 모델 정의
class KoBARTWithLongformer(BartForConditionalGeneration):
    def __init__(self, config):
        super().__init__(config)

        if config.attention_mode == 'n2':
            pass
        else:
            # 포지셔널 임베딩 확장 (encoder)
            self.model.encoder.embed_positions = BartLearnedPositionalEmbedding(
                config.max_encoder_position_embeddings,  # num_embeddings
                config.d_model  # embedding_dim
            )

            # 인코더에 Longformer Attention 적용
            for i, encoder_layer in enumerate(self.model.encoder.layers):
                encoder_layer.self_attn = LongformerAttention(config, layer_id=i)

In [4]:
# Longformer Attention 적용 KoBART 설정
class KoBARTConfig(BartConfig):
    def __init__(self, attention_window: List[int] = None,
                 attention_dilation: List[int] = None,
                 autoregressive: bool = False,
                 attention_mode: str = 'sliding_chunks',
                 gradient_checkpointing: bool = False,
                 attention_probs_dropout_prob: float = 0.1,
                 **kwargs):
        super().__init__(**kwargs)
        self.attention_window = attention_window
        self.attention_dilation = attention_dilation
        self.autoregressive = autoregressive
        self.attention_mode = attention_mode
        self.gradient_checkpointing = gradient_checkpointing
        self.attention_probs_dropout_prob = attention_probs_dropout_prob

        assert self.attention_mode in ['sliding_chunks', 'n2']

In [11]:
# 모델 초기화 함수
def initialize_kobart_with_longformer():
    attention_window_size = 512  # 짝수
    max_position_embeddings = 4104  # 1026의 배수

    # KoBART 기본 설정
    config = KoBARTConfig(
        vocab_size=30000,  # KoBART 기본 vocab 크기
        d_model=768,  # Hidden Dimension
        encoder_layers=6,  # 인코더 레이어 수
        decoder_layers=6,  # 디코더 레이어 수
        encoder_attention_heads=16,  # Attention Head 수
        decoder_attention_heads=16,  # Attention Head 수
        encoder_ffn_dim=3072,  # FFN 차원
        decoder_ffn_dim=3072,  # FFN 차원
        attention_window=[attention_window_size] * 6,  # Longformer Attention Window 설정
        max_encoder_position_embeddings=max_position_embeddings,  # 확장된 포지셔널 임베딩
        max_decoder_position_embeddings=1026,  # 디코더 크기 유지
        attention_mode='sliding_chunks',
        gradient_checkpointing=True,
        attention_probs_dropout_prob=0.1  # 드롭아웃 설정
    )

    # Tokenizer 및 모델 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("gogamza/kobart-base-v2")
    model = KoBARTWithLongformer.from_pretrained(
        "gogamza/kobart-base-v2",
        config=config,
        ignore_mismatched_sizes=True  # 초기 로드 시 불일치 무시
    )

    # 인코더 포지셔널 임베딩 확장
    current_max_pos, embed_size = model.model.encoder.embed_positions.weight.shape
    print(f"Current encoder max pos: {current_max_pos}")
    new_encoder_pos_embed = model.model.encoder.embed_positions.weight.new_empty(
        config.max_encoder_position_embeddings + 2, embed_size
    )
    # 기존 가중치 복사
    new_encoder_pos_embed[:current_max_pos] = model.model.encoder.embed_positions.weight
    # 확장된 부분 랜덤 초기화
    nn.init.normal_(new_encoder_pos_embed[current_max_pos:], mean=0.0, std=0.02)
    model.model.encoder.embed_positions.weight.data = new_encoder_pos_embed

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    return model, tokenizer

In [9]:
# 모델과 토크나이저 초기화
model, tokenizer = initialize_kobart_with_longformer()
print("KoBART with Longformer Attention is ready!")

tokenizer_config.json:   0%|          | 0.00/39.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Some weights of KoBARTWithLongformer were not initialized from the model checkpoint at EbanLee/kobart-summary-v3 and are newly initialized: ['model.encoder.layers.0.self_attn.longformer_self_attn.key.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.key.weight', 'model.encoder.layers.0.self_attn.longformer_self_attn.key_global.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.key_global.weight', 'model.encoder.layers.0.self_attn.longformer_self_attn.query.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.query.weight', 'model.encoder.layers.0.self_attn.longformer_self_attn.query_global.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.query_global.weight', 'model.encoder.layers.0.self_attn.longformer_self_attn.value.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.value.weight', 'model.encoder.layers.0.self_attn.longformer_self_attn.value_global.bias', 'model.encoder.layers.0.self_attn.longformer_self_attn.value_global.weight

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Current encoder max pos: 4106
KoBART with Longformer Attention is ready!


In [10]:
# 샘플 데이터 정의
sample_input = """
한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학자 이호왕 고려대 명예교수.
고려대의대 제공 한탄바이러스를 발견한 우리나라 대표 의과학자 이호왕 고려대 명예교수가 5일 숙환으로 별세했다.
향년 94세. 고인은 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을 대표하는 의학자이자 미생물학자다.
신증후군출혈열 병원체인 한탄바이러스와 서울바이러스를 세계 최초로 발견하고 예방백신 및 진단법을 개발해 세계 의학발전에 기여한 것으로 평가된다.
1928년 함경남도 신흥에서 출생한 고인은 1973년 고대의대에 부임해 의과대학장을 지냈으며 1982년 세계보건기구 신증후출혈열연구협력센터 소장 2000년 대한민국학술원 회장 등을 역임했다.
1979년 미국 최고민간인공로훈장 1987년 인촌상 1992년 호암상 1995년 태국 프린스 마히돌상 2001년 일본 니케이 아시아상 2002년 과학기술훈장 창조장 2009년 서재필의학상
2018년 대한민국 과학기술유공자로 추대됐으며 2002년 미국 학술원 NAS 외국회원 2009년 일본 학사원 명예회원에 선정되며 국내외 학계에서 활발한 활동을 이어왔다.
고인의 빈소는 고려대 안암병원 장례식장 303호에 마련됐다. 발인은 7일 오전 11시 50분이고 장지는 서울추모공원이다.
"""

# 모델을 통해 요약 생성 테스트
def test_model_on_sample(model, tokenizer, input_text, max_length=1024, summary_length=50):
    # 입력 텍스트 토큰화
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=max_length,
        truncation=True,
        padding="max_length"
    )

    # 입력을 모델로 전달하여 요약 생성
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    # 모델 디코딩
    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=summary_length,
        num_beams=4,  # 빔 서치 적용
        no_repeat_ngram_size=2,  # 반복 방지
        early_stopping=True  # 조기 종료
    )

    # 생성된 요약 디코딩
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary


# 모델 성능 테스트
summary = test_model_on_sample(model, tokenizer, sample_input)
print(f"Input: {sample_input.strip()}")
print(f"Generated Summary: {summary}")

Input: 한탄바이러스 발견 노벨상 유력 후보로 자주 거론 한국을 대표하는 의학자이자 미생물학자 이호왕 고려대 명예교수.
고려대의대 제공 한탄바이러스를 발견한 우리나라 대표 의과학자 이호왕 고려대 명예교수가 5일 숙환으로 별세했다.
향년 94세. 고인은 바이러스의 병원체와 진단법 백신까지 모두 개발한 한국을 대표하는 의학자이자 미생물학자다.
신증후군출혈열 병원체인 한탄바이러스와 서울바이러스를 세계 최초로 발견하고 예방백신 및 진단법을 개발해 세계 의학발전에 기여한 것으로 평가된다.
1928년 함경남도 신흥에서 출생한 고인은 1973년 고대의대에 부임해 의과대학장을 지냈으며 1982년 세계보건기구 신증후출혈열연구협력센터 소장 2000년 대한민국학술원 회장 등을 역임했다.
1979년 미국 최고민간인공로훈장 1987년 인촌상 1992년 호암상 1995년 태국 프린스 마히돌상 2001년 일본 니케이 아시아상 2002년 과학기술훈장 창조장 2009년 서재필의학상
2018년 대한민국 과학기술유공자로 추대됐으며 2002년 미국 학술원 NAS 외국회원 2009년 일본 학사원 명예회원에 선정되며 국내외 학계에서 활발한 활동을 이어왔다.
고인의 빈소는 고려대 안암병원 장례식장 303호에 마련됐다. 발인은 7일 오전 11시 50분이고 장지는 서울추모공원이다.
Generated Summary: 다.음돈/품·3식·출조사하다.이다.설며착색일악며,답갑선법
